In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('/home/saloni/Documents/ML/Mortality/train.csv')
labels=pd.read_csv('/home/saloni/Documents/ML/Mortality/labels_train.csv')


In [4]:
dailydata=df.copy()
result=pd.concat([dailydata,labels],axis=1)

In [5]:
result.head(1)

,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,DiasABP,...,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,In-hospital_death
0,77.0,31.0,46.0,54,2.973333,10.5,0.7,154.0,0.75,58.795833,...,97.25,116.891892,37.357143,2.1,0.14,171.052632,10.3,80.060976,7.387273,0


In [6]:
list_of_variables = ['Age', 'Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol', 'Creatinine',
                             'DiasABP', 'FiO2', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'Height', 'HR', 'ICUType', 'K',
                             'Lactate', 'Mg', 'MAP', 'MechVent', 'Na', 'NIDiasABP', 'NIMAP', 'NISysABP', 'PaCO2', 'PaO2',
                             'pH', 'Platelets', 'RecordID', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI','TroponinT','Urine',
                             'WBC', 'Weight']

In [7]:
datawithoutoutliers=result.copy()
    

In [8]:
for i in list_of_variables:
    datawithoutoutliers=datawithoutoutliers[np.abs(datawithoutoutliers[i]-datawithoutoutliers[i].mean())<=(4*datawithoutoutliers[i].std())]
    outliers=datawithoutoutliers[np.abs(datawithoutoutliers[i]-datawithoutoutliers[i].mean())>=(4*datawithoutoutliers[i].std())]
    
print(datawithoutoutliers.shape)

(3169, 43)


In [9]:
labels=datawithoutoutliers.pop("In-hospital_death")
df=datawithoutoutliers.copy()

In [10]:
labels.shape


(3169,)

In [11]:
df.shape

(3169, 42)

In [12]:
from sklearn.preprocessing import MinMaxScaler
#normalization
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)


In [13]:
from imblearn.over_sampling import SMOTE

df_res,labels_res=SMOTE(random_state=9).fit_sample(df,labels) #balancing the data
df_res=pd.DataFrame(df_res)
labels_res=pd.DataFrame(labels_res)

In [14]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)
onehot_encoded = enc.fit_transform(labels_res)

labels_res=onehot_encoded

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_res,labels_res,test_size=0.33,random_state=42)


In [16]:
sess=tf.InteractiveSession()

In [17]:
n_hidden_1=60
n_hidden_2=60
n_hidden_3=60
n_hidden_4=60
n_hidden_5=60


x=tf.placeholder(tf.float32,shape=[None,42])
y_=tf.placeholder(tf.float32,shape=[None,2])
w =tf.Variable(tf.zeros([42,2]))
b=tf.Variable(tf.zeros([2]))

def multilayer_perceptron(x,weights,biases):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.sigmoid(layer_3)
    
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.sigmoid(layer_4)
    
    layer_5=tf.add(tf.matmul(layer_4,weights['h5']),biases['b5'])
    layer_5=tf.nn.relu(layer_5)
    
    
    out_layer=tf.add(tf.matmul(layer_5,weights['out']),biases['out'])
    return out_layer

weights={
       
    'h1':tf.Variable(tf.truncated_normal([42,n_hidden_1])),
    'h2':tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3':tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4':tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
    'h5':tf.Variable(tf.truncated_normal([n_hidden_4,n_hidden_5])),
    'out':tf.Variable(tf.truncated_normal([n_hidden_5,2]))
}

biases={
    'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
    'b5':tf.Variable(tf.truncated_normal([n_hidden_5])),
    'out':tf.Variable(tf.truncated_normal([2]))
}
    
y=multilayer_perceptron(x,weights,biases)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y))  
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
sess.run(tf.global_variables_initializer())                     
                     
    
    

In [18]:
mse_history=[]
cost_history=[]
accuracy_history=[]

In [19]:
training_epochs=1000

In [20]:
for epoch in range(training_epochs):
    sess.run(train_step,feed_dict={x:x_train,y_:y_train})
    cost=sess.run(cross_entropy,feed_dict={x:x_train,y_:y_train})
    cost_history.append(np.append(cost_history,cost))
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    pred_y=sess.run(y,feed_dict={x:x_test})
    mse=tf.reduce_mean(tf.square(pred_y-y_test))
    mse_=sess.run(mse)
    mse_history.append(mse_)
    accuracy=sess.run(accuracy,feed_dict={x:x_train,y_:y_train})
    accuracy_history.append(accuracy)
    
    print('epoch:',epoch,'-','cost:',cost,'mse: ',mse_,'-','-train accuracy:',accuracy)

epoch: 0 - cost: 4.13997 mse:  23.0395371825 - -train accuracy: 0.507852
epoch: 1 - cost: 3.36464 mse:  16.470584732 - -train accuracy: 0.491083
epoch: 2 - cost: 4.02057 mse:  22.1605490725 - -train accuracy: 0.507852
epoch: 3 - cost: 3.26303 mse:  15.9814954779 - -train accuracy: 0.490551
epoch: 4 - cost: 3.90712 mse:  21.3599912802 - -train accuracy: 0.507852
epoch: 5 - cost: 3.18228 mse:  15.6339458892 - -train accuracy: 0.490285
epoch: 6 - cost: 3.80241 mse:  20.6500972006 - -train accuracy: 0.507852
epoch: 7 - cost: 3.11775 mse:  15.3812176505 - -train accuracy: 0.490019
epoch: 8 - cost: 3.71088 mse:  20.0508240843 - -train accuracy: 0.507852
epoch: 9 - cost: 3.06137 mse:  15.1774259407 - -train accuracy: 0.489486
epoch: 10 - cost: 3.62867 mse:  19.5326504279 - -train accuracy: 0.507852
epoch: 11 - cost: 3.01166 mse:  15.0109660611 - -train accuracy: 0.488954
epoch: 12 - cost: 3.55403 mse:  19.0684070257 - -train accuracy: 0.507852
epoch: 13 - cost: 2.96834 mse:  14.877117109 - -t

epoch: 113 - cost: 2.04683 mse:  10.2018393729 - -train accuracy: 0.492148
epoch: 114 - cost: 2.11726 mse:  10.533943857 - -train accuracy: 0.508917
epoch: 115 - cost: 2.03706 mse:  10.1245182543 - -train accuracy: 0.492414
epoch: 116 - cost: 2.10572 mse:  10.4564802944 - -train accuracy: 0.509183
epoch: 117 - cost: 2.02754 mse:  10.0491555465 - -train accuracy: 0.492414
epoch: 118 - cost: 2.09482 mse:  10.3844182708 - -train accuracy: 0.509183
epoch: 119 - cost: 2.01812 mse:  9.97598454539 - -train accuracy: 0.49268
epoch: 120 - cost: 2.08391 mse:  10.3125759316 - -train accuracy: 0.508917
epoch: 121 - cost: 2.00852 mse:  9.9031901039 - -train accuracy: 0.492947
epoch: 122 - cost: 2.07317 mse:  10.2423971304 - -train accuracy: 0.508917
epoch: 123 - cost: 1.99941 mse:  9.83254705888 - -train accuracy: 0.492947
epoch: 124 - cost: 2.06278 mse:  10.175064847 - -train accuracy: 0.508917
epoch: 125 - cost: 1.9902 mse:  9.76429609048 - -train accuracy: 0.492947
epoch: 126 - cost: 2.05245 mse

epoch: 223 - cost: 1.63641 mse:  7.66283667428 - -train accuracy: 0.499601
epoch: 224 - cost: 1.65407 mse:  8.01642374724 - -train accuracy: 0.509715
epoch: 225 - cost: 1.63018 mse:  7.63211584389 - -train accuracy: 0.499601
epoch: 226 - cost: 1.64751 mse:  7.98707568926 - -train accuracy: 0.509981
epoch: 227 - cost: 1.62401 mse:  7.60137136526 - -train accuracy: 0.499601
epoch: 228 - cost: 1.64113 mse:  7.95766792604 - -train accuracy: 0.509981
epoch: 229 - cost: 1.61784 mse:  7.57056917821 - -train accuracy: 0.500399
epoch: 230 - cost: 1.63477 mse:  7.92865292319 - -train accuracy: 0.509981
epoch: 231 - cost: 1.61183 mse:  7.54155656993 - -train accuracy: 0.501198
epoch: 232 - cost: 1.62841 mse:  7.90141135904 - -train accuracy: 0.510248
epoch: 233 - cost: 1.60569 mse:  7.51301067641 - -train accuracy: 0.501198
epoch: 234 - cost: 1.622 mse:  7.87438854647 - -train accuracy: 0.510248
epoch: 235 - cost: 1.59967 mse:  7.48521588701 - -train accuracy: 0.50173
epoch: 236 - cost: 1.6157 ms

epoch: 333 - cost: 1.34758 mse:  6.37076599421 - -train accuracy: 0.51078
epoch: 334 - cost: 1.35791 mse:  6.76924208852 - -train accuracy: 0.520362
epoch: 335 - cost: 1.34306 mse:  6.35444203094 - -train accuracy: 0.510514
epoch: 336 - cost: 1.3533 mse:  6.75287327166 - -train accuracy: 0.520628
epoch: 337 - cost: 1.3385 mse:  6.33726813012 - -train accuracy: 0.51078
epoch: 338 - cost: 1.34884 mse:  6.7364259979 - -train accuracy: 0.52116
epoch: 339 - cost: 1.334 mse:  6.32030167765 - -train accuracy: 0.51078
epoch: 340 - cost: 1.34439 mse:  6.72050137385 - -train accuracy: 0.521427
epoch: 341 - cost: 1.3295 mse:  6.30347961627 - -train accuracy: 0.51078
epoch: 342 - cost: 1.33989 mse:  6.70415650651 - -train accuracy: 0.521427
epoch: 343 - cost: 1.32504 mse:  6.2859675388 - -train accuracy: 0.51078
epoch: 344 - cost: 1.3355 mse:  6.68776683772 - -train accuracy: 0.522225
epoch: 345 - cost: 1.32067 mse:  6.27006426575 - -train accuracy: 0.511312
epoch: 346 - cost: 1.33104 mse:  6.6717

epoch: 443 - cost: 1.13314 mse:  5.61671691671 - -train accuracy: 0.525685
epoch: 444 - cost: 1.14426 mse:  6.01467834515 - -train accuracy: 0.534203
epoch: 445 - cost: 1.12974 mse:  5.60694834735 - -train accuracy: 0.526218
epoch: 446 - cost: 1.14088 mse:  6.0050486914 - -train accuracy: 0.534469
epoch: 447 - cost: 1.12637 mse:  5.5979432235 - -train accuracy: 0.526484
epoch: 448 - cost: 1.13753 mse:  5.9958650694 - -train accuracy: 0.535534
epoch: 449 - cost: 1.12297 mse:  5.58856809045 - -train accuracy: 0.526484
epoch: 450 - cost: 1.13423 mse:  5.98665553563 - -train accuracy: 0.536332
epoch: 451 - cost: 1.1196 mse:  5.57945253937 - -train accuracy: 0.52675
epoch: 452 - cost: 1.13097 mse:  5.97760669748 - -train accuracy: 0.536598
epoch: 453 - cost: 1.11626 mse:  5.57052671011 - -train accuracy: 0.527016
epoch: 454 - cost: 1.12765 mse:  5.96773281837 - -train accuracy: 0.536598
epoch: 455 - cost: 1.11295 mse:  5.56092737928 - -train accuracy: 0.527016
epoch: 456 - cost: 1.12443 mse

epoch: 553 - cost: 0.97244 mse:  5.14440295073 - -train accuracy: 0.541123
epoch: 554 - cost: 0.984251 mse:  5.49604719741 - -train accuracy: 0.559755
epoch: 555 - cost: 0.969998 mse:  5.13728794219 - -train accuracy: 0.542188
epoch: 556 - cost: 0.981784 mse:  5.48731051602 - -train accuracy: 0.561086
epoch: 557 - cost: 0.967575 mse:  5.12991124913 - -train accuracy: 0.542188
epoch: 558 - cost: 0.979382 mse:  5.47812624629 - -train accuracy: 0.561618
epoch: 559 - cost: 0.965156 mse:  5.12199565787 - -train accuracy: 0.542188
epoch: 560 - cost: 0.976983 mse:  5.46983967614 - -train accuracy: 0.562417
epoch: 561 - cost: 0.962776 mse:  5.11507513752 - -train accuracy: 0.542986
epoch: 562 - cost: 0.974566 mse:  5.46158518434 - -train accuracy: 0.562683
epoch: 563 - cost: 0.960346 mse:  5.1077820189 - -train accuracy: 0.543253
epoch: 564 - cost: 0.972122 mse:  5.45314823278 - -train accuracy: 0.562683
epoch: 565 - cost: 0.957919 mse:  5.10046341118 - -train accuracy: 0.543253
epoch: 566 - c

epoch: 662 - cost: 0.867171 mse:  5.07210310404 - -train accuracy: 0.590897
epoch: 663 - cost: 0.854463 mse:  4.78874735262 - -train accuracy: 0.562683
epoch: 664 - cost: 0.865313 mse:  5.06421215527 - -train accuracy: 0.591962
epoch: 665 - cost: 0.852661 mse:  4.78258813265 - -train accuracy: 0.562949
epoch: 666 - cost: 0.863474 mse:  5.05639322033 - -train accuracy: 0.59276
epoch: 667 - cost: 0.850841 mse:  4.77643272317 - -train accuracy: 0.563215
epoch: 668 - cost: 0.861609 mse:  5.04886175106 - -train accuracy: 0.593825
epoch: 669 - cost: 0.84903 mse:  4.77044127229 - -train accuracy: 0.56428
epoch: 670 - cost: 0.859752 mse:  5.04132491457 - -train accuracy: 0.594357
epoch: 671 - cost: 0.847226 mse:  4.76425158401 - -train accuracy: 0.565078
epoch: 672 - cost: 0.857914 mse:  5.03379390777 - -train accuracy: 0.595688
epoch: 673 - cost: 0.84548 mse:  4.75875204074 - -train accuracy: 0.565877
epoch: 674 - cost: 0.856097 mse:  5.02659549114 - -train accuracy: 0.595954
epoch: 675 - cos

epoch: 771 - cost: 0.76946 mse:  4.50932158874 - -train accuracy: 0.585041
epoch: 772 - cost: 0.778296 mse:  4.7119734913 - -train accuracy: 0.620442
epoch: 773 - cost: 0.768068 mse:  4.50514362265 - -train accuracy: 0.586106
epoch: 774 - cost: 0.776819 mse:  4.70625419724 - -train accuracy: 0.620974
epoch: 775 - cost: 0.766669 mse:  4.50073928236 - -train accuracy: 0.587703
epoch: 776 - cost: 0.775375 mse:  4.70044889332 - -train accuracy: 0.622039
epoch: 777 - cost: 0.765288 mse:  4.49614447636 - -train accuracy: 0.587969
epoch: 778 - cost: 0.773944 mse:  4.69480434534 - -train accuracy: 0.623104
epoch: 779 - cost: 0.76394 mse:  4.49158877024 - -train accuracy: 0.587969
epoch: 780 - cost: 0.772563 mse:  4.68898655387 - -train accuracy: 0.623104
epoch: 781 - cost: 0.762603 mse:  4.48702817128 - -train accuracy: 0.588768
epoch: 782 - cost: 0.77117 mse:  4.68320137719 - -train accuracy: 0.623902
epoch: 783 - cost: 0.76123 mse:  4.48261986262 - -train accuracy: 0.589832
epoch: 784 - cost

epoch: 880 - cost: 0.712017 mse:  4.45160675695 - -train accuracy: 0.644131
epoch: 881 - cost: 0.704352 mse:  4.30224216874 - -train accuracy: 0.612723
epoch: 882 - cost: 0.710937 mse:  4.44732522631 - -train accuracy: 0.644397
epoch: 883 - cost: 0.703327 mse:  4.29875259024 - -train accuracy: 0.612989
epoch: 884 - cost: 0.709863 mse:  4.44294433761 - -train accuracy: 0.644663
epoch: 885 - cost: 0.702305 mse:  4.29521028217 - -train accuracy: 0.613255
epoch: 886 - cost: 0.70879 mse:  4.43856322692 - -train accuracy: 0.644929
epoch: 887 - cost: 0.701266 mse:  4.29164399051 - -train accuracy: 0.614852
epoch: 888 - cost: 0.707727 mse:  4.43416679349 - -train accuracy: 0.645728
epoch: 889 - cost: 0.700244 mse:  4.28809797743 - -train accuracy: 0.615118
epoch: 890 - cost: 0.706667 mse:  4.42965268001 - -train accuracy: 0.645994
epoch: 891 - cost: 0.699249 mse:  4.28451644607 - -train accuracy: 0.614852
epoch: 892 - cost: 0.705643 mse:  4.4253978021 - -train accuracy: 0.64626
epoch: 893 - co

epoch: 989 - cost: 0.657264 mse:  4.15200002041 - -train accuracy: 0.628693
epoch: 990 - cost: 0.662036 mse:  4.25741684092 - -train accuracy: 0.670482
epoch: 991 - cost: 0.65652 mse:  4.14955305052 - -train accuracy: 0.628959
epoch: 992 - cost: 0.661268 mse:  4.25436774608 - -train accuracy: 0.671014
epoch: 993 - cost: 0.655794 mse:  4.14707179704 - -train accuracy: 0.629492
epoch: 994 - cost: 0.660511 mse:  4.25113945165 - -train accuracy: 0.671546
epoch: 995 - cost: 0.655064 mse:  4.14452916256 - -train accuracy: 0.629492
epoch: 996 - cost: 0.659792 mse:  4.24812603948 - -train accuracy: 0.671546
epoch: 997 - cost: 0.65438 mse:  4.14215823065 - -train accuracy: 0.629758
epoch: 998 - cost: 0.659068 mse:  4.24505112758 - -train accuracy: 0.672345
epoch: 999 - cost: 0.653683 mse:  4.13977063811 - -train accuracy: 0.629492


In [21]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('test accuracy:',sess.run(accuracy,feed_dict={x:x_test,y_:y_test}))

test accuracy: 0.633171


In [22]:
pred_y=sess.run(y,feed_dict={x:x_test})
mse=tf.reduce_mean(tf.square(pred_y-y_test))
print('mse:%.4f'%sess.run(mse))

mse:4.1398
